# Experiments

In [6]:
# Imports
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import defaultdict
import transformers
import gensim
import json
import os

import experiments

In [7]:
# lists
high_threat_list = ['afperser', 'agent', 'agente', 'arrestant', 'arrestanten', 'autodief', 'autokraker', 'bajesklant', 'bandiet', 'bandieten', 'bankovervaller', 'bankrover', 'bedelaar', 'bedreiger', 'bende', 'bendeleden', 'bendeleider', 'bendelid', 'benden', 'bendes', 'beroepscrimineel', 'berovingen', 'beschieting', 'beul', 'boef', 'bolletjesslikker', 'bolletjesslikkers', 'bommenmaker', 'bordelen', 'brandstichter', 'brandstichters', 'corrupt', 'criminaliteit', 'crimineel', 'criminelen', 'cyberpesten', 'dader', 'daders', 'delinquent', 'delinquenten', 'dief', 'draaideurcrimineel', 'drugsbaas', 'drugsbaron', 'drugsbende', 'drugsbendes', 'drugscrimineel', 'drugsdealer', 'drugsdealers', 'drugsgebruikers', 'drugshandelaar', 'drugshandelaars', 'drugssmokkelaar', 'dubbelagent', 'fietsendief', 'gangster', 'gangsterbende', 'gedetineerde', 'gedetineerden', 'gegijzelden', 'gevangenbewaarders', 'gevangene', 'gevangenen', 'gevangenisbewaarder', 'gevangenissen', 'geweldsman', 'gijzelaar', 'gijzelaars', 'gijzelnemer', 'gijzelnemers', 'handlanger', 'hardrijder', 'hoofdagent', 'hoofdagente', 'hoofddader', 'hoofdverdachte', 'hooligan', 'hooligans', 'huurmoord', 'huurmoordenaar', 'illegalen', 'inbreker', 'indringer', 'jeugdbende', 'jeugdbendes', 'jeugddelinquent', 'kaper', 'kapers', 'kidnapper', 'kidnappers', 'kinderlokker', 'kindermisbruiker', 'kindermoordenaar', 'kindslaven', 'krijgsgevangenen', 'kruimeldief', 'kunstdief', 'ladykiller', 'lastpak', 'lastpost', 'liquidatie', 'loverboy', 'lovergirls', 'lustmoordenaar', 'maffia', 'maffiabaas', 'maffiosi', 'maffioso', 'maffiabaas', 'massamoordenaar', 'massamoordenaars', 'mededader', 'medegedetineerde', 'medegevangene', 'medeplichtige', 'medeverdachte', 'mensenhandelaren', 'mensensmokkelaar', 'mensensmokkelaars', 'messentrekker', 'misdaden', 'misdadig', 'misdadiger', 'misdadigers', 'wapenhandelaar', 'moordenaar', 'moordenaars', 'moordmachine', 'moordverdachte', 'motoragent', 'neerstak', 'neersteken', 'ontvoeringen', 'oplichter', 'overvaller', 'pedofiel', 'pedofielen', 'piraten', 'plunderaar', 'plunderaars', 'politieagent', 'politieagente', 'politieagenten', 'politiecommandant', 'politiegeneraal', 'politiegewonde', 'politieman', 'politiemannen', 'politiemensen', 'politieofficier', 'politiepost', 'politierechercheur', 'politiestaat', 'politievrouw', 'politiemensen', 'pyromaan', 'recidivist', 'relschopper', 'relschoppers', 'roofmoord', 'roofoverval', 'scherpschutter', 'schutter', 'seriemoordenaar', 'skinheads', 'slaaf', 'slachtoffers', 'slaven', 'sluipschutter', 'sluipschutters', 'smokkelaar', 'smokkelaars', 'snelheidsduivel', 'souteneur', 'stalker', 'straatbende', 'strafbaar', 'strafklacht', 'struikrover', 'tasjesdief', 'terreurgroep', 'terreurverdachte', 'terrorist', 'uitbuiting', 'vechtersbaas', 'veelpleger', 'veelplegers', 'veiligheidsagent', 'veiligheidsagenten', 'veiligheidspolitie', 'verdachte', 'verkrachter', 'vermisten', 'voortvluchtige', 'vreemdeling', 'vrouwenhandelaar', 'wapenhandelaar', 'winkeldief', 'winkeldievegge', 'wreker', 'wurgmoord', 'zakkenrollers', 'zedendelinquent', 'zelfmoordenaar', 'zwartrijder']
low_status_list = ['achterlijke', 'achterlijk', 'armoedig', 'achterstanden', 'achterstandskinderen', 'achterstandsleerling', 'achterstandsleerlingen', 'achterstandswijken', 'achterstelling', 'alcoholicus', 'alcoholist', 'alcoholiste', 'alcoholisten', 'analfabeet', 'analfabete', 'analfabeten', 'barbaars', 'bastaardzoon', 'bedelaar', 'bedelaars', 'bijstandsgerechtigden', 'bijstandsgerechtigde', 'boerenlul', 'dakloze', 'daklozen', 'dronkenlap', 'druggebruikers', 'drugsgebruiker', 'drugsgebruikers', 'drugsrunners', 'drugstoeristen', 'drugsverslaafde', 'drugsverslaafden', 'hangjongere', 'hangjongeren', 'hoer', 'hoerenlopers', 'hulpbehoevend', 'hulpbehoevende', 'idioot', 'junk', 'junkies', 'junks', 'kansarme', 'kansarmen', 'kindertehuizen', 'krottenwijk', 'laaggeschoolde', 'laaggeschoolden', 'laagopgeleide', 'laagopgeleiden', 'loser', 'malloot', 'minderwaardig', 'nestbevuiler', 'nietsnut', 'onderklasse', 'onderontwikkeld', 'ongeletterde', 'ongeschoolde', 'overlastgevende', 'pooier', 'primitief', 'probleemjongeren', 'prostituee', 'prostituees', 'prostitutiebedrijven', 'reljongeren', 'schoolverlaters', 'slet', 'sloeber', 'sloebers', 'spijbelaar', 'spijbelen', 'straatarm', 'straatkinderen', 'straatprostitutie', 'sukkel', 'taalachterstand', 'tienermoeder', 'tienermoeders', 'uitkeringsgerechtigde', 'uitkeringsgerechtigden', 'uitwas', 'verschoppelingen', 'verslaafde', 'verslaafden', 'weeskinderen', 'werkloos', 'werkloze', 'werklozen', 'werkschuwe', 'zwerver', 'zwervers']
etniciteiten_ev = [
        'nederlander',
        'duitser',
        'belg',
        'pool',
        'surinamer',
        'turk',
        'syrier',
        'antilliaan',
        'somalier',
        'afghaan',
        'irakees',
        'marokkaan'
        ]
etniciteiten_mv = [
        'nederlanders',
        'duitsers',
        'belgen',
        'polen',
        'surinamers',
        'turken',
        'syriers',
        'antillianen',
        'somaliers',
        'afghanen',
        'irakezen',
        'marokkanen'
        ]
in_group = ['nederlander', 'belg', 'duitser']
out_group = ['pool', 'surinamer', 'turk', 'syrier', 'antilliaan', 'somalier', 'afghaan', 'irakees', 'marokkaan']

# Word2Vec

# Experiment 1
Experiment 1 of the Word2Vec analysis involves generating a list of the top 100 most similar words for each ethnicity, in both the singular and plural form (‘Marokkaan' and ‘Marokkanen'). The resulting list is then used to determine the proportion of words that appear in the high-threat and low-status stereotype word lists for each ethnicity. This approach facilitates a comparison of the extent to which the most similar words align with terms commonly associated with high-threat and low-status stereotypes across different ethnicities. Moreover, the analysis examines whether the ethnic outgroup exhibits a greater similarity to high-threat and low-status stereotype words relative to the ethnic ingroup, by averaging the total proportion of most similar words appearing in the high-threat and low-status list of the ethnic ingroup and outgroup.

In [8]:
# load model
model = gensim.models.Word2Vec.load('./word2vec_AEM/word2vec_model_AEM.model')

In [9]:
# Write the top 100 most similar words to file

with open('word2vec_results.csv', 'a') as wf:
    wf.write(f'etniciteit, ')
    for i in range(100):
        wf.write(f'{1+i}, ')
    wf.write('\n')
    
for etn in etniciteiten_ev+etniciteiten_mv:
    try:
        top100 = model.wv.most_similar(positive=[etn], topn=100)
    except:
        print(f'{etn} not in vocabulary! Maybe different spelling?')
        continue
    line = ''
    line+=f'{etn}, '
    for word in top100:
        line += f'{word[0]}, '
    with open('word2vec_results.csv', 'a') as wf:
        wf.write(f'{line}\n')

In [10]:
# Count occurences of high_threat, low_status words in top 100
# IMPORTANT : Ev & Mv already combined here

w2v_high_threat = defaultdict(list)
w2v_low_status = defaultdict(list)

for ev, mv in zip(etniciteiten_ev, etniciteiten_mv):
    top100_ev = model.wv.most_similar(positive=[ev], topn=100)
    top100_mv = model.wv.most_similar(positive=[mv], topn=100)
    top100_words = [i for i,_ in top100_ev]+[i for i,_ in top100_mv]

    for i, word in enumerate(top100_words):
        if word in high_threat_list:
            w2v_high_threat[ev].append(top100_words[i])

for ev, mv in zip(etniciteiten_ev, etniciteiten_mv):
    top100_ev = model.wv.most_similar(positive=[ev], topn=100)
    top100_mv = model.wv.most_similar(positive=[mv], topn=100)
    top100_words = [i for i,_ in top100_ev]+[i for i,_ in top100_mv]

    for i, word in enumerate(top100_words):
        if word in low_status_list:
            w2v_low_status[ev].append(top100_words[i])

In [11]:
# Occurences of high threat - low status words per etnicity
# if etnicity not printed, 0 words have occurred. 
for key, value in w2v_high_threat.items():
    print(f'{key} has {len(value)} occurences of high threat words')

print('\n')

for key, value in w2v_low_status.items():
    print(f'{key} has {len(value)} occurences of low status words')

nederlander has 1 occurences of high threat words
duitser has 1 occurences of high threat words
belg has 1 occurences of high threat words
pool has 2 occurences of high threat words
surinamer has 10 occurences of high threat words
turk has 13 occurences of high threat words
syrier has 5 occurences of high threat words
antilliaan has 23 occurences of high threat words
somalier has 24 occurences of high threat words
afghaan has 31 occurences of high threat words
irakees has 38 occurences of high threat words
marokkaan has 25 occurences of high threat words


surinamer has 9 occurences of low status words
antilliaan has 17 occurences of low status words
somalier has 1 occurences of low status words
marokkaan has 3 occurences of low status words


In [12]:
import plotly.express as px
etnicity_mapping = {
    'nederlander' : 'Dutch',
    'duitser' : 'German',
    'belg' : 'Belgian',
    'pool' : 'Polish',
    'syrier' : 'Syrian',
    'surinamer' : 'Surinamese',
    'turk' : 'Turkish',
    'antilliaan' : 'Antillean',
    'somalier' : 'Somali',
    'marokkaan' : 'Moroccan',
    'afghaan' : 'Afghan',
    'irakees' : 'Iraqi'
}

ht_count_df = pd.DataFrame(columns=['Ethnicity', 'Count', 'Ethnic Group'])
for i, (key, value) in enumerate(w2v_high_threat.items()):
    if key in in_group:
        ht_count_df.loc[i] = [etnicity_mapping[key], len(value), 'In']
    else:
        ht_count_df.loc[i] = [etnicity_mapping[key], len(value), 'Out']

ht_count_df.sort_values(by='Count', ascending=True, inplace=True)
fig = px.bar(ht_count_df, x='Ethnicity', y='Count', color='Ethnic Group', title='Amount of High-Threat Occurences per Ethnicity', width=1000, height=400, text_auto=True)
fig.show()

In [13]:
ht_count_df.sort_values(by='Count', ascending=True, inplace=True)
fig = px.histogram(ht_count_df, x='Ethnic Group', y='Count', color='Ethnic Group', title='Total Amount of High-Threat Occurrences for Ethnic In- and Outgroups', width=1000, height=400, text_auto=True)
fig.update_yaxes(title_text='Sum of Occurrences')
fig.show()

In [14]:
low_status_counts = {
    'nederlander' : [],
    'duitser' : [],
    'belg' : [],
    'pool' : [],
    'syrier' : [],
    'surinamer' : [],
    'turk' : [],
    'antilliaan' : [],
    'somalier' : [],
    'marokkaan' : [],
    'afghaan' : [],
    'irakees' : []
}
low_status_counts.update(w2v_low_status)

ls_count_df = pd.DataFrame(columns=['Etnicity', 'Count', 'Ethnic Group'])
for i, (key, value) in enumerate(low_status_counts.items()):
    if key in in_group:
        ls_count_df.loc[i] = [etnicity_mapping[key], len(value), 'In']
    else:
        ls_count_df.loc[i] = [etnicity_mapping[key], len(value), 'Out']

ls_count_df.sort_values(by='Count', ascending=True, inplace=True)
fig = px.bar(ls_count_df, x='Etnicity', y='Count', color='Ethnic Group', title='Amount of Low-Status Occurences per Ethnicity', width=1000, height=400, text_auto=True)
fig.show()

fig = px.histogram(ls_count_df, x='Ethnic Group', y='Count', color='Ethnic Group',title='Total Amount of Low-Status Occurrences for Ethnic In- and Outgroups', width=1000, height=400, text_auto=True)
fig.update_yaxes(title_text='Sum of Occurrences')
fig.show()

In [15]:
# Occurences of in-out group
in_group_ht_occ = 0
out_group_ht_occ = 0

in_group_ls_occ = 0
out_group_ls_occ = 0

for etn in in_group:
    if w2v_high_threat.get(etn):
        in_group_ht_occ += len(w2v_high_threat.get(etn))

    if w2v_low_status.get(etn):
        in_group_ls_occ += len(w2v_low_status.get(etn))

for etn in out_group:
    if w2v_high_threat.get(etn):
        out_group_ht_occ += len(w2v_high_threat.get(etn))

    if w2v_low_status.get(etn):
        out_group_ls_occ += len(w2v_low_status.get(etn))


print(f'in-group high threat occurences {in_group_ht_occ}')
print(f'out-group high threat occurences {out_group_ht_occ}')
print(f'in-group low status occurences {in_group_ls_occ}')
print(f'out-group low status occurences {out_group_ls_occ}')

in-group high threat occurences 3
out-group high threat occurences 171
in-group low status occurences 0
out-group low status occurences 30


# Experiment 2

Experiment 2 of the Word2Vec analysis assesses the degree to which ethnicities are associated with words in the high-threat and low-status stereotype word lists. This is achieved by calculating the association strength between all ethnicities (in both singular and plural forms) and all words in the high-threat and low-status lists. The average association strength is then computed for each ethnicity, which allows for a comparison of how strongly each ethnicity is linked to high-threat and low-status stereotypes. Finally, the average association strength is calculated separately for the ethnic ingroup and outgroup to determine whether there are any differences in the extent to which each group is associated with ethnic stereotypes.

In [16]:
# get association scores

association_dict_high_threat = defaultdict(list)

for target, (ev, mv) in zip(high_threat_list, zip(etniciteiten_ev, etniciteiten_mv)):
    target = target.strip()
    ev = ev.strip()
    mv = mv.strip()
    if not model.wv.__contains__(target):
        continue
    if not model.wv.__contains__(ev):
        continue
    if not model.wv.__contains__(mv):
        continue
    ass_ev = model.wv.similarity(target, ev)
    ass_mv = model.wv.similarity(target, mv)

    association_dict_high_threat[ev].extend([ass_ev, ass_mv])

association_dict_low_status = defaultdict(list)

for target, (ev, mv) in zip(low_status_list, zip(etniciteiten_ev, etniciteiten_mv)):
    target = target.strip()
    ev = ev.strip()
    mv = mv.strip()
    if not model.wv.__contains__(target):
        continue
    if not model.wv.__contains__(ev):
        continue
    if not model.wv.__contains__(mv):
        continue
    ass_ev = model.wv.similarity(target, ev)
    ass_mv = model.wv.similarity(target, mv)

    association_dict_low_status[ev].extend([ass_ev, ass_mv])

# ev + mv
high_threat_ass_score = {}
low_status_ass_score = {}

for ev in etniciteiten_ev:
    high_threat_ass_score[ev] = np.mean(association_dict_high_threat[ev])
    low_status_ass_score[ev] = np.mean(association_dict_low_status[ev])

print(high_threat_ass_score)
print(low_status_ass_score)

{'nederlander': 0.26571813, 'duitser': 0.25653413, 'belg': 0.12190824, 'pool': 0.21532185, 'surinamer': 0.38295352, 'turk': 0.3341215, 'syrier': 0.32552463, 'antilliaan': 0.41241485, 'somalier': 0.4315082, 'afghaan': 0.5705559, 'irakees': 0.38837844, 'marokkaan': 0.3636636}
{'nederlander': 0.020933537, 'duitser': 0.022767354, 'belg': -0.060468163, 'pool': -0.0047139004, 'surinamer': 0.3810606, 'turk': 0.24456164, 'syrier': 0.10464919, 'antilliaan': 0.29809555, 'somalier': 0.16082919, 'afghaan': 0.15455663, 'irakees': 0.32342696, 'marokkaan': 0.16050552}


In [17]:
ht_ass_df = pd.DataFrame(columns=['Etnicity', 'Similarity Score', 'Ethnic Group'])
for i, (key, value) in enumerate(high_threat_ass_score.items()):
    if key in in_group:
        ht_ass_df.loc[i] = [etnicity_mapping[key], round(value, 3), 'in']
    else:
        ht_ass_df.loc[i] = [etnicity_mapping[key], round(value, 3), 'out']

ht_ass_df.sort_values(by='Similarity Score', ascending=True, inplace=True)
fig = px.bar(ht_ass_df, x='Etnicity', y='Similarity Score', color='Ethnic Group', title='High-Threat Similarity Score per Ethnicity', width=1000, height=400, text_auto=True)
fig.show()

fig = px.histogram(ht_ass_df, x='Ethnic Group', y='Similarity Score', color='Ethnic Group',title='Average High-Threat Similarity Score per Ethnic Group', width=1000, height=400, text_auto=True, histfunc='avg')
fig.show()

In [18]:
ls_ass_df = pd.DataFrame(columns=['Etnicity', 'Similarity Score', 'Ethnic Group'])
for i, (key, value) in enumerate(low_status_ass_score.items()):
    if key in in_group:
        ls_ass_df.loc[i] = [etnicity_mapping[key], round(value, 3), 'in']
    else:
        ls_ass_df.loc[i] = [etnicity_mapping[key], round(value, 3), 'out']

ls_ass_df.sort_values(by='Similarity Score', ascending=True, inplace=True)
fig = px.bar(ls_ass_df, x='Etnicity', y='Similarity Score', color='Ethnic Group', title='Low-Status Similarity Score per Ethnicity', width=1000, height=400, text_auto=True)
fig.show()

fig = px.histogram(ls_ass_df, x='Ethnic Group', y='Similarity Score', color='Ethnic Group',title='Average Low-Status Similarity Score per Ethnic Group', width=1000, height=400, text_auto=True, histfunc='avg')
fig.show()

In [19]:
# in-out group association scores

in_group_high_threat_association = []
out_group_high_threat_association = []
in_group_low_status_association = []
out_group_low_status_association = []

for etn in in_group:
    in_group_high_threat_association.append(high_threat_ass_score[etn])
    in_group_low_status_association.append(low_status_ass_score[etn])

for etn in out_group:
    out_group_high_threat_association.append(high_threat_ass_score[etn])
    out_group_low_status_association.append(low_status_ass_score[etn])

print(f'in group high threat association : {np.mean(in_group_high_threat_association)}')
print(f'out group high threat association : {np.mean(out_group_high_threat_association)}')
print(f'in group low status association : {np.mean(in_group_low_status_association)}')
print(f'out group low status association : {np.mean(out_group_low_status_association)}')

in group high threat association : 0.21472017467021942
out group high threat association : 0.3804936110973358
in group low status association : -0.005589091684669256
out group low status association : 0.20255237817764282


# Bertje


The experiments reported below are conducted both on the original BERTje model developed by de Vries et al. (2019) and on the additionally trained BERTje model, so that the inherent ethnic bias in the BERTje model and the bias created by additionally training on the corpus of Dutch news can be examined, respectively.

In [20]:
# Load data from files

model = 'FINETUNED_2-6'
with open(f'./results/{model}_high_threat_probs_ev.txt', 'r') as rf:
    ft_high_threat_probs_ev = json.load(rf)

with open(f'./results/{model}_high_threat_probs_mv.txt', 'r') as rf:
    ft_high_threat_probs_mv = json.load(rf)

with open(f'./results/{model}_low_status_probs_ev.txt', 'r') as rf:
    ft_low_status_probs_ev = json.load(rf)

with open(f'./results/{model}_low_status_probs_mv.txt', 'r') as rf:
    ft_low_status_probs_mv = json.load(rf)

with open(f'./results/{model}_top100_ev.txt', 'r') as rf:
    ft_fill_mask_dict_ev = json.load(rf)

with open(f'./results/{model}_top100_mv.txt', 'r') as rf:
    ft_fill_mask_dict_mv = json.load(rf)

with open(f'./results/GroNLP_high_threat_probs_ev.txt', 'r') as rf:
    nlp_high_threat_probs_ev = json.load(rf)

with open(f'./results/GroNLP_high_threat_probs_mv.txt', 'r') as rf:
    nlp_high_threat_probs_mv = json.load(rf)

with open(f'./results/GroNLP_low_status_probs_ev.txt', 'r') as rf:
    nlp_low_status_probs_ev = json.load(rf)

with open(f'./results/GroNLP_low_status_probs_mv.txt', 'r') as rf:
    nlp_low_status_probs_mv = json.load(rf)

with open(f'./results/GroNLP_top100_ev.txt', 'r') as rf:
    nlp_fill_mask_dict_ev = json.load(rf)

with open(f'./results/GroNLP_top100_mv.txt', 'r') as rf:
    nlp_fill_mask_dict_mv = json.load(rf)

In [ ]:
for key, value in nlp_fill_mask_dict_ev.items():
    nlp_fill_mask_dict_ev[key] = [v.lower() for v in value]

for key, value in nlp_fill_mask_dict_mv.items():
    nlp_fill_mask_dict_mv[key] = [v.lower() for v in value]

for key, value in ft_fill_mask_dict_ev.items():
    ft_fill_mask_dict_ev[key] = [v.lower() for v in value]

for key, value in ft_fill_mask_dict_mv.items():
    ft_fill_mask_dict_mv[key] = [v.lower() for v in value]

print(nlp_fill_mask_dict_ev)

# Experiment 1 (target-neutral)

As previously mentioned, this experiment involves constructing two templates (one for singular and one for plural forms) of neutral sentences containing a mask. The singular form of ethnicities is represented by the sentence "The (ethnicity singular form) is a [MASK]," while the plural form of ethnicities is captured by the sentence "The (ethnicities plural form) are [MASK].” The model predicts the top 100 masks for each template, and the resulting list is then analyzed to determine how many of the predicted masks are categorized under the high-threat and low-status stereotype word lists. This analysis enables the calculation of the percentage/amount of ethnically stereotypical predicted masks for each ethnicity.

In [21]:
# count occurrences of high threat low status
ft_high_threat_counts = {}
ft_low_status_counts = {}

for ev, mv in zip(etniciteiten_ev, etniciteiten_mv):
    ft_high_threat_counts[ev] = len(set(high_threat_list).intersection(set(ft_fill_mask_dict_ev[ev]))) + len(set(high_threat_list).intersection(set(ft_fill_mask_dict_mv[mv])))
    ft_low_status_counts[ev] = len(set(low_status_list).intersection(set(ft_fill_mask_dict_ev[ev]))) + len(set(low_status_list).intersection(set(ft_fill_mask_dict_mv[mv])))

nlp_high_threat_counts = {}
nlp_low_status_counts = {}

for ev, mv in zip(etniciteiten_ev, etniciteiten_mv):
    nlp_high_threat_counts[ev] = len(set(high_threat_list).intersection(set(nlp_fill_mask_dict_ev[ev]))) + len(set(high_threat_list).intersection(set(nlp_fill_mask_dict_mv[mv])))
    nlp_low_status_counts[ev] = len(set(low_status_list).intersection(set(nlp_fill_mask_dict_ev[ev]))) + len(set(low_status_list).intersection(set(nlp_fill_mask_dict_mv[mv])))

print(ft_high_threat_counts)
print(ft_low_status_counts)

print(nlp_high_threat_counts)
print(nlp_low_status_counts)

{'nederlander': 0, 'duitser': 0, 'belg': 0, 'pool': 0, 'surinamer': 0, 'turk': 0, 'syrier': 2, 'antilliaan': 0, 'somalier': 1, 'afghaan': 0, 'irakees': 1, 'marokkaan': 0}
{'nederlander': 0, 'duitser': 0, 'belg': 0, 'pool': 0, 'surinamer': 0, 'turk': 0, 'syrier': 0, 'antilliaan': 0, 'somalier': 0, 'afghaan': 0, 'irakees': 0, 'marokkaan': 0}
{'nederlander': 1, 'duitser': 1, 'belg': 0, 'pool': 0, 'surinamer': 0, 'turk': 0, 'syrier': 3, 'antilliaan': 4, 'somalier': 0, 'afghaan': 1, 'irakees': 1, 'marokkaan': 1}
{'nederlander': 2, 'duitser': 1, 'belg': 0, 'pool': 0, 'surinamer': 0, 'turk': 0, 'syrier': 1, 'antilliaan': 2, 'somalier': 1, 'afghaan': 0, 'irakees': 1, 'marokkaan': 1}


In [22]:
import plotly.graph_objects as go

bert_ht_count_df = pd.DataFrame(columns=['Ethnicity', 'AdditionallyTrainedBERTje', 'OriginalBERTje', 'EthnicGroup'])
for i, etn in enumerate(ft_high_threat_counts.keys()):
    if etn in in_group:
        bert_ht_count_df.loc[i] = [etnicity_mapping[etn], ft_high_threat_counts[etn], nlp_high_threat_counts[etn], 'In']
    else:
       bert_ht_count_df.loc[i] = [etnicity_mapping[etn], ft_high_threat_counts[etn], nlp_high_threat_counts[etn], 'Out']

bert_ht_count_df.sort_values(by='AdditionallyTrainedBERTje', ascending=True, inplace=True)

fig = go.Figure()

fig.add_trace(go.Bar(
    x=bert_ht_count_df.Ethnicity,
    y=bert_ht_count_df.AdditionallyTrainedBERTje,
    name='Additionally Trained BERTje'
))
fig.add_trace(go.Bar(
    x=bert_ht_count_df.Ethnicity,
    y=bert_ht_count_df.OriginalBERTje,
    name='Original BERTje'
))

fig.update_layout(
    title='Amount of Predicted High-Threat Tokens per Ethnicity and Ethnic Group',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Amount',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
)
fig.show()

In [23]:
import plotly.graph_objects as go

bert_ht_count_df = pd.DataFrame(columns=['Ethnicity', 'AdditionallyTrainedBERTje', 'OriginalBERTje', 'EthnicGroup'])
for i, etn in enumerate(ft_high_threat_counts.keys()):
    if etn in in_group:
        bert_ht_count_df.loc[i] = [etnicity_mapping[etn], ft_high_threat_counts[etn], nlp_high_threat_counts[etn], 'In']
    else:
        bert_ht_count_df.loc[i] = [etnicity_mapping[etn], ft_high_threat_counts[etn], nlp_high_threat_counts[etn], 'Out']

bert_ht_count_df.sort_values(by='AdditionallyTrainedBERTje', ascending=True, inplace=True)

fig1 = go.Figure()
fig1.add_trace(go.Bar(
    x=bert_ht_count_df.Ethnicity,
    y=bert_ht_count_df.AdditionallyTrainedBERTje,
    name='Additionally Trained'
))
fig1.update_layout(
    title='High Threat Counts - Additionally Trained BERTje',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Sum of Occurrences',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    bargap=0.15,  # gap between bars of adjacent location coordinates.
    bargroupgap=0.1  # gap between bars of the same location coordinate.
)

fig2 = go.Figure()
fig2.add_trace(go.Bar(
    x=bert_ht_count_df.Ethnicity,
    y=bert_ht_count_df.OriginalBERTje,
    name='Original BERTje'
))
fig2.update_layout(
    title='High Threat Counts - Original BERTje',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Sum of Occurrences',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    bargap=0.15,  # gap between bars of adjacent location coordinates.
    bargroupgap=0.1  # gap between bars of the same location coordinate.
)

fig1.show()
fig2.show()

In [24]:
bert_ls_count_df = pd.DataFrame(columns=['Ethnicity', 'AdditionallyTrainedBERTje', 'OriginalBERTje', 'EthnicGroup'])
for i, etn in enumerate(ft_low_status_counts.keys()):
    if etn in in_group:
        bert_ls_count_df.loc[i] = [etnicity_mapping[etn], ft_low_status_counts[etn], nlp_low_status_counts[etn], 'in']
    else:
       bert_ls_count_df.loc[i] = [etnicity_mapping[etn], ft_low_status_counts[etn], nlp_low_status_counts[etn], 'out']

bert_ls_count_df.sort_values(by='AdditionallyTrainedBERTje', ascending=True, inplace=True)

fig = go.Figure()

fig.add_trace(go.Bar(
    x=bert_ls_count_df.Ethnicity,
    y=bert_ls_count_df.AdditionallyTrainedBERTje,
    name='AdditionallyTrainedBERTje'
))
fig.add_trace(go.Bar(
    x=bert_ls_count_df.Ethnicity,
    y=bert_ls_count_df.OriginalBERTje,
    name='OriginalBERTje'
))

fig.update_layout(
    title='Amount of Predicted Low-Status Tokens per Ethnicity and Ethnic Group',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Amount',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
)
fig.show()

In [25]:
import plotly.graph_objects as go

bert_ls_count_df = pd.DataFrame(columns=['Ethnicity', 'AdditionallyTrainedBERTje', 'OriginalBERTje', 'EthnicGroup'])
for i, etn in enumerate(ft_low_status_counts.keys()):
    if etn in in_group:
        bert_ls_count_df.loc[i] = [etnicity_mapping[etn], ft_low_status_counts[etn], nlp_low_status_counts[etn], 'in']
    else:
       bert_ls_count_df.loc[i] = [etnicity_mapping[etn], ft_low_status_counts[etn], nlp_low_status_counts[etn], 'out']

bert_ls_count_df.sort_values(by='AdditionallyTrainedBERTje', ascending=True, inplace=True)

fig1 = go.Figure()
fig1.add_trace(go.Bar(
    x=bert_ls_count_df.Ethnicity,
    y=bert_ls_count_df.AdditionallyTrainedBERTje,
    name='Additionally Trained BERTje'
))
fig1.update_layout(
    title='Amount of Predicted Low-Status Tokens - Additionally Trained BERTje',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Amount of Tokens',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    bargap=0.15,  # gap between bars of adjacent location coordinates.
    bargroupgap=0.1  # gap between bars of the same location coordinate.
)

fig2 = go.Figure()
fig2.add_trace(go.Bar(
    x=bert_ls_count_df.Ethnicity,
    y=bert_ls_count_df.OriginalBERTje,
    name='Original BERTje'
))
fig2.update_layout(
    title='Amount of Predicted Low-Status Tokens - Original BERTje',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Amount of Tokens',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    bargap=0.15,  # gap between bars of adjacent location coordinates.
    bargroupgap=0.1  # gap between bars of the same location coordinate.
)

fig1.show()
fig2.show()

# Experiment 2 (Target-Attribute).

The latter experiment involves the creation of template sentences that incorporate the target ethnicity and high-threat and low-status attributes. These attributes encompass various word types, such as nouns and adjectives, thus requiring the generation of different types of masked sentences to accommodate all potential attribute words. For each sentence, the ethnicity mask is replaced with all ethnicities (both singular and plural), and the model computes the probability of the sentence containing the high-threat and low-status words. This process yields an average probability score for each ethnicity across all sentences with high-threat and low-status words. Using these scores, the mean probability can be calculated for both high-threat and low-status stereotypes within the ethnic ingroup and ethnic outgroup.

In [26]:
# Generate dict with the ev+mv association scores
ft_high_threat_association = {}
ft_low_status_association = {}
nlp_high_threat_association = {}
nlp_low_status_association = {}

# finetuned
for ev, mv in zip(ft_high_threat_probs_ev.keys(), ft_high_threat_probs_mv.keys()):
    ft_high_threat_association[ev] = np.mean([np.mean([i for j, i in ft_high_threat_probs_ev[ev]]), np.mean([i for _, i in ft_high_threat_probs_mv[mv]])])
    ft_low_status_association[ev] = np.mean([np.mean([i for _, i in ft_low_status_probs_ev[ev]]), np.mean([i for _, i in ft_low_status_probs_mv[mv]])])

# Bertje
for ev, mv in zip(nlp_high_threat_probs_ev.keys(), nlp_high_threat_probs_mv.keys()):
    nlp_high_threat_association[ev] = np.mean([np.mean([i for j, i in nlp_high_threat_probs_ev[ev]]), np.mean([i for _, i in nlp_high_threat_probs_mv[mv]])])
    nlp_low_status_association[ev] = np.mean([np.mean([i for _, i in nlp_low_status_probs_ev[ev]]), np.mean([i for _, i in nlp_low_status_probs_mv[mv]])])

for etn in ft_high_threat_association.keys():
    print(f'{etn} high threat scores : finetuned = {ft_high_threat_association[etn]}, bertje = {nlp_high_threat_association[etn]}')
    print(f'{etn} low status scores : finetuned = {ft_low_status_association[etn]}, bertje = {nlp_low_status_association[etn]}')

nederlander high threat scores : finetuned = 0.0005306681278024664, bertje = 0.00974398007550293
nederlander low status scores : finetuned = 0.00011778291575749184, bertje = 0.009017453995810646
duitser high threat scores : finetuned = 0.0006914882221134809, bertje = 0.010757930401242238
duitser low status scores : finetuned = 0.00011193682481758688, bertje = 0.012184753727990022
belg high threat scores : finetuned = 0.0005610545002895014, bertje = 0.011362369600268672
belg low status scores : finetuned = 5.144559314741162e-05, bertje = 0.01273057484343143
pool high threat scores : finetuned = 0.0006721089697346925, bertje = 0.009880261570512461
pool low status scores : finetuned = 0.00010968109377436572, bertje = 0.010096243432057863
surinamer high threat scores : finetuned = 0.0008634131706998878, bertje = 0.01102814399364543
surinamer low status scores : finetuned = 0.00015553232790107465, bertje = 0.01156945178236532
turk high threat scores : finetuned = 0.0011154667461037988, bert

In [27]:
# in-out group association scores
ft_in_group_high_threat_association = []
ft_out_group_high_threat_association = []
ft_in_group_low_status_association = []
ft_out_group_low_status_association = []

for etn in in_group:
    ft_in_group_high_threat_association.append(ft_high_threat_association[etn])
    ft_in_group_low_status_association.append(ft_low_status_association[etn])

for etn in out_group:
    ft_out_group_high_threat_association.append(ft_high_threat_association[etn])
    ft_out_group_low_status_association.append(ft_low_status_association[etn])

nlp_in_group_high_threat_association = []
nlp_out_group_high_threat_association = []
nlp_in_group_low_status_association = []
nlp_out_group_low_status_association = []

for etn in in_group:
    nlp_in_group_high_threat_association.append(nlp_high_threat_association[etn])
    nlp_in_group_low_status_association.append(nlp_low_status_association[etn])

for etn in out_group:
    nlp_out_group_high_threat_association.append(nlp_high_threat_association[etn])
    nlp_out_group_low_status_association.append(nlp_low_status_association[etn])

print(f'finetuned in group high threat association : {np.mean(ft_in_group_high_threat_association)}')
print(f'finetuned out group high threat association : {np.mean(ft_out_group_high_threat_association)}')
print(f'finetuned in group low status association : {np.mean(ft_in_group_low_status_association)}')
print(f'finetuned out group low status association : {np.mean(ft_out_group_low_status_association)}')
print('\n')
print(f'bertje in group high threat association : {np.mean(nlp_in_group_high_threat_association)}')
print(f'bertje out group high threat association : {np.mean(nlp_out_group_high_threat_association)}')
print(f'bertje in group low status association : {np.mean(nlp_in_group_low_status_association)}')
print(f'bertje out group low status association : {np.mean(nlp_out_group_low_status_association)}')
print('\n')
print(f'finetuned high threat association ratio out-group/in-group : {np.mean(ft_out_group_high_threat_association)/np.mean(ft_in_group_high_threat_association)}')
print(f'finetuned low status association ratio out-group/in-group : {np.mean(ft_out_group_low_status_association)/np.mean(ft_in_group_low_status_association)}')
print(f'bertje high threat association ratio out-group/in-group : {np.mean(nlp_out_group_high_threat_association)/np.mean(nlp_in_group_high_threat_association)}')
print(f'bertje low status association ratio out-group/in-group : {np.mean(nlp_out_group_low_status_association)/np.mean(nlp_in_group_low_status_association)}')

finetuned in group high threat association : 0.0005944036167351496
finetuned out group high threat association : 0.0016761843836005522
finetuned in group low status association : 9.372177790749678e-05
finetuned out group low status association : 0.0001629064712373068


bertje in group high threat association : 0.010621426692337947
bertje out group high threat association : 0.011327326228643462
bertje in group low status association : 0.011310927522410698
bertje out group low status association : 0.010791779800469642


finetuned high threat association ratio out-group/in-group : 2.819943110049102
finetuned low status association ratio out-group/in-group : 1.7381922843812798
bertje high threat association ratio out-group/in-group : 1.0664599546513591
bertje low status association ratio out-group/in-group : 0.9541021086986498


In [28]:
bert_ht_ass_df = pd.DataFrame(columns=['Ethnicity', 'Additionally Trained BERTje', 'Original BERTje', 'Ethnic Group'])
for i, etn in enumerate(ft_high_threat_counts.keys()):
    if etn in in_group:
        bert_ht_ass_df.loc[i] = [etnicity_mapping[etn], ft_high_threat_association[etn], nlp_high_threat_association[etn], 'In']
    else:
       bert_ht_ass_df.loc[i] = [etnicity_mapping[etn], ft_high_threat_association[etn], nlp_high_threat_association[etn], 'Out']

bert_ht_ass_df.sort_values(by='Additionally Trained BERTje', ascending=True, inplace=True)

fig = px.bar(bert_ht_ass_df, x='Ethnicity', y='Additionally Trained BERTje', color='Ethnic Group', title='Additionally Trained BERTje: Average High-Threat Probabilities per Ethnicity')
fig.show()

fig = px.bar(bert_ht_ass_df, x='Ethnicity', y='Original BERTje', color='Ethnic Group', title='Original BERTje: Average High-Threat Probabilities per Ethnicity')
fig.show()

fig = px.histogram(bert_ht_ass_df, x='Ethnic Group', y='Additionally Trained BERTje', color='Ethnic Group', title='Additionally Trained BERTje Average High-Threat Probabilities per Ethnic Group', histfunc='avg')
fig.show()

fig = px.histogram(bert_ht_ass_df, x='Ethnic Group', y='Original BERTje', color='Ethnic Group', title='Original BERTje: Average High-Threat Probabilities per Ethnic Group', histfunc='avg')
fig.show()



In [29]:
bert_ls_ass_df = pd.DataFrame(columns=['Ethnicity', 'Additionally Trained BERTje', 'Original BERTje', 'Ethnic Group'])
for i, etn in enumerate(ft_low_status_counts.keys()):
    if etn in in_group:
        bert_ls_ass_df.loc[i] = [etnicity_mapping[etn], ft_low_status_association[etn], nlp_low_status_association[etn], 'In']
    else:
       bert_ls_ass_df.loc[i] = [etnicity_mapping[etn], ft_low_status_association[etn], nlp_low_status_association[etn], 'Out']

bert_ls_ass_df.sort_values(by='Additionally Trained BERTje', ascending=True, inplace=True)

fig = px.bar(bert_ls_ass_df, x='Ethnicity', y='Additionally Trained BERTje', color='Ethnic Group', title='Additionally Trained BERTje: Average Low-Status Probabilities per Ethnicity')
fig.update_yaxes(title='Average Probability')
fig.show()

fig = px.bar(bert_ls_ass_df, x='Ethnicity', y='Original BERTje', color='Ethnic Group', title='Original BERTje: Average Low-Status Probabilities per Ethnicity')
fig.update_yaxes(title='Average Probability')
fig.show()

fig = px.histogram(bert_ls_ass_df, x='Ethnic Group', y='Additionally Trained BERTje', color='Ethnic Group', title='Additionally Trained BERTje: Average Low-Status Probabilities per Ethnic Group', histfunc='avg')
fig.update_yaxes(title='Average Probability')
fig.show()

fig = px.histogram(bert_ls_ass_df, x='Ethnic Group', y='Original BERTje', color='Ethnic Group', title='Original BERTje: Average Low-Status Probabilities per Ethnic Group', histfunc='avg')
fig.update_yaxes(title='Average Probability')
fig.show()